# Extract turbine quantities

In [1]:
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['/projects/wind_uq/lcheung/amrwind-frontend/',
                  '/ccs/proj/cfd162/lcheung/amrwind-frontend/',
                ]
import sys, os, shutil, io
import numpy as np
for x in amrwindfedirs: sys.path.insert(1, x)

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import glob

# Make all plots inline 
#%matplotlib inline

import postproengine as ppeng

/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# Load ruamel or pyyaml as needed
try:
    import ruamel.yaml
    yaml = ruamel.yaml.YAML(typ='unsafe', pure=True)
    useruamel=True
    loaderkwargs = {'Loader':yaml.RoundTripLoader}
    dumperkwargs = {'Dumper':yaml.RoundTripDumper, 'indent':4, 'default_flow_style':False}
    Loader=yaml.load
except:
    import yaml as yaml
    useruamel=False
    loaderkwargs = {}
    dumperkwargs = {'default_flow_style':False }
    Loader=yaml.safe_load
    
def stringReplaceDict(s, dreplace):
    outstr = str(s)
    for k, g in dreplace.items():
        outstr=outstr.replace(k, str(g))
    return outstr
    

In [3]:
replacedict={'RUNDIRA':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/',
             'RUNDIR_A1':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1/',
             'RUNDIR_A2':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2/',
             'RUNDIR_A3':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3/',
             'RUNDIRB':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runB/',
             'RUNDIRC':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runC/',
             'RESULTSDIRA':'OFRESULTS_RUNA',
             'RESULTSDIR_A1':'OFRESULTS_RUNA1',
             'RESULTSDIR_A2':'OFRESULTS_RUNA2',
             'RESULTSDIR_A3':'OFRESULTS_RUNA3',
             'RESULTSDIRB':'OFRESULTS_RUNB',
             'RESULTSDIRC':'OFRESULTS_RUNC',
            }

In [4]:
yamlstring="""
globalattributes:
  verbose: False
  executeorder:
  - openfast
  
openfast:

# RUNDIRA
- name: [] #[T0, T1]
  filename: []
  #- RUNDIRA/WIT_USKPL_SS001_WT001_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.out
  #- RUNDIRA/WIT_USKPL_SS001_WT002_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.T1.out
  vars: 
  - Time
  - NacYaw
  - GenPwr
  - Wind1VelX
  - Wind1VelY
  - Wind1VelZ
  output_dir: RESULTSDIRA
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False

# RUNDIR_A1
- name: [] 
  filename: []
  vars: 
  - Time
  - NacYaw
  - GenPwr
  - Wind1VelX
  - Wind1VelY
  - Wind1VelZ
  output_dir: RESULTSDIR_A1
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False
    
# RUNDIR_A2
- name: [] 
  filename: []
  vars: 
  - Time
  - NacYaw
  - GenPwr
  - Wind1VelX
  - Wind1VelY
  - Wind1VelZ
  output_dir: RESULTSDIR_A2
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False

# RUNDIR_A3
- name: [] 
  filename: []
  vars: 
  - Time
  - NacYaw
  - GenPwr
  - Wind1VelX
  - Wind1VelY
  - Wind1VelZ
  output_dir: RESULTSDIR_A3
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False

Junk:
# RUNDIRB
- name: [] 
  filename: []
  vars: 
  - Time
  - NacYaw
  - GenPwr
  output_dir: RESULTSDIRB
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False
    
# RUNDIRC
- name: [] 
  filename: []
  vars: 
  - Time
  - NacYaw
  - GenPwr
  output_dir: RESULTSDIRC
  useregex: True
  csv:  # Store information to CSV files
    individual_files: False

#junk:


"""
f = io.StringIO(stringReplaceDict(yamlstring, replacedict))
yamldict = Loader(f, **loaderkwargs)

In [5]:
for irun, run in enumerate(['RUNDIRA',  'RUNDIR_A1',  'RUNDIR_A2', 'RUNDIR_A3', ]):  #enumerate(['RUNDIRA',  'RUNDIR_A1',  'RUNDIR_A2', 'RUNDIRB', 'RUNDIRC']): #
    turblist = glob.glob(replacedict[run]+'/WIT_USKPL_SS001_WT0??_OpenFAST_NREL2p8-127_AWAKEN/*.out')
    #print(turblist)
    for t in turblist:
        for d in t.split('/'):
            if d.startswith('WIT'):
                tname = d.replace('_OpenFAST_NREL2p8-127_AWAKEN','')
        #print(tname, t)
        yamldict['openfast'][irun]['name'].append(tname)
        yamldict['openfast'][irun]['filename'].append(t)

In [6]:
# Run the driver
ppeng.driver(yamldict, verbose=True)

Initialized openfast
Running openfast
WIT_USKPL_SS001_WT046 /tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/WIT_USKPL_SS001_WT046_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.T43.out
Initialized csv inside WIT_USKPL_SS001_WT046
Executing csv
WIT_USKPL_SS001_WT007 /tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/WIT_USKPL_SS001_WT007_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.T6.out
Initialized csv inside WIT_USKPL_SS001_WT007
Executing csv
WIT_USKPL_SS001_WT073 /tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/WIT_USKPL_SS001_WT073_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.T70.out
Initialized csv inside WIT_USKPL_SS001_WT073
Executing csv
WIT_USKPL_SS001_WT062 /tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/WIT_USKPL_SS001_WT062_OpenFAST_NREL2p8-127_AWAKEN/NREL-2p8-127.T59.out
Initialized csv inside WIT_USKPL_SS001_WT062
Executing csv
WIT_USKPL_SS001_WT004 /tscratch/lcheung/AWAKEN/Benchmark1/P